In [1]:
import sys
import torch
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import time
import os
import numpy as np
import json
import cv2
from PIL import Image, ImageOps
import random
from tqdm import tqdm
import operator
import itertools
from scipy.io import  loadmat
import logging
from scipy import signal
from utils import data_transforms
from tqdm import tqdm

from models.gazenet import GazeNet
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.gazenet import GooDataset, GazeDataset

In [2]:
def boxes2centers(normalized_boxes):
    
    center_x = (normalized_boxes[:,0] + normalized_boxes[:,2]) / 2
    center_y = (normalized_boxes[:,1] + normalized_boxes[:,3]) / 2
    center_x = np.expand_dims(center_x, axis=1)
    center_y = np.expand_dims(center_y, axis=1)
    normalized_centers = np.hstack((center_x, center_y))
    
    return normalized_centers

def select_nearest_bbox(gazepoint, gt_bboxes, gt_labels):
    '''
    In: Accepts gazepoint (2,) and bboxes (n_boxes, 4), normalized from [0,1]
    Out: Returns the bbox nearest to gazepoint.
    '''
    
    centers = boxes2centers(gt_bboxes)
    
    diff = centers - gazepoint
    l2dist = np.sqrt(diff[:,0]**2 + diff[:,1]**2)
    min_idx = l2dist.argmin()
    
    
    nearest_box = {
        'box' : gt_bboxes[min_idx],
        'label': gt_labels[min_idx],
        'index' : min_idx
    }
    return nearest_box

def calculate_metrics(npzfile, dataset):
    predictions = np.load(npzfile)
    
    error = []
    percent_dists=[0.01, 0.03, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
    PA_count = np.zeros((len(percent_dists)))
    CPA_count = np.zeros((len(percent_dists)))
    
    for idx, f_point in tqdm(enumerate(predictions['gazepoints']), total=len(dataset)):
        
        data = dataset[idx]
        
        # Calculate L2, and use for pa/cpa
        gt_point = data['gt_position'].numpy()
        f_error = f_point - gt_point
        f_dist = np.sqrt(f_error[0] ** 2 + f_error[1] ** 2)
        error.append(f_dist)
        
        #Calc pa
        PA_count[np.array(percent_dists) > f_dist] += 1   
        
        #Calc cpa
        gt_bboxes, gt_labels, gaze_class = data['gt_bboxes'], data['gt_labels'], data['gaze_class']
        #gt_bboxes = gt_bboxes / [640, 480, 640, 480]
        selected_bbox = select_nearest_bbox(f_point, gt_bboxes, gt_labels)
        if selected_bbox['label'] == gaze_class: 
            CPA_count[np.array(percent_dists) > f_dist] += 1   
        
        
    mse = np.mean(np.array(error))
    pa = PA_count / len(dataset)
    cpa = CPA_count / len(dataset)
    metrics = {
        'mse' : mse,
        'pa' : pa,
        'cpa': cpa
    }
    
    return metrics

In [3]:
def draw_result(image_path, eye, heatmap, gaze_point, gt_point, idx=0):
    x1, y1 = eye
    x2, y2 = gaze_point
    x3, y3 = gt_point
    im = cv2.imread(image_path)
    image_height, image_width = im.shape[:2]
    x1, y1 = image_width * x1, y1 * image_height
    x2, y2 = image_width * x2, y2 * image_height
    x3, y3 = image_width * x3, y3 * image_height
    x1, y1, x2, y2, x3, y3 = map(int, [x1, y1, x2, y2, x3, y3])
    cv2.circle(im, (x1, y1), 5, [255, 255, 255], -1)
    cv2.circle(im, (x2, y2), 5, [255, 255, 255], -1)
    cv2.circle(im, (x3, y3), 5, [255, 255, 255], -1)
    cv2.line(im, (x1, y1), (x2, y2), [255, 0, 0], 2)
    cv2.line(im, (x1, y1), (x3, y3), [0, 165, 255], 2)

    # heatmap visualization
    heatmap = ((heatmap - heatmap.min()) / (heatmap.max() - heatmap.min()) * 255).astype(np.uint8)
    heatmap = np.stack([heatmap, heatmap, heatmap], axis=2)
    heatmap = cv2.resize(heatmap, (image_width, image_height))

    heatmap = (0.8 * heatmap.astype(np.float32) + 0.2 * im.astype(np.float32)).astype(np.uint8)
    img = np.concatenate((im, heatmap), axis=1)
    
    save_dir = './sample_out/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    filename = 'out_%s.png' % str(idx)
    save_path = save_dir + filename
    print(save_path)
    cv2.imwrite(save_path, img)

    return img

In [4]:
def test_and_save(net, test_data_loader, outfile='predictions.npz'):
    net.eval()

    all_heatmaps = []
    all_gazepoints = []
    with torch.no_grad():
        for data in tqdm(test_data_loader, total=len(test_data_loader)):
            image, face_image, gaze_field, eye_position, gt_position, gt_heatmap = \
                data['image'], data['face_image'], data['gaze_field'], data['eye_position'], data['gt_position'], data['gt_heatmap']
            image, face_image, gaze_field, eye_position, gt_position, gt_heatmap = \
                map(lambda x: Variable(x.cuda()), [image, face_image, gaze_field, eye_position, gt_position, gt_heatmap])

            direction, predict_heatmap = net([image, face_image, gaze_field, eye_position])

            final_output = predict_heatmap.cpu().data.numpy()
            target = gt_position.cpu().data.numpy()
            all_heatmaps.append(final_output)

            for f_point, gt_point, eye_point in \
                zip(final_output, target, eye_position):
                f_point = f_point.reshape([224 // 4, 224 // 4])

                h_index, w_index = np.unravel_index(f_point.argmax(), f_point.shape)
                f_point = np.array([w_index / 56., h_index / 56.])             
                all_gazepoints.append(f_point)

                f_error = f_point - gt_point
                f_dist = np.sqrt(f_error[0] ** 2 + f_error[1] ** 2)

    all_heatmaps = np.vstack(all_heatmaps).squeeze()
    all_gazepoints = np.vstack(all_gazepoints)
    np.savez(outfile, heatmaps=all_heatmaps, gazepoints=all_gazepoints)
    
    return

In [5]:
# Load Model
net = GazeNet()
net.cuda()

resume_path = './saved_models/gazenet_goo/model_epoch25.pth.tar'
net, optimizer, start_epoch = resume_checkpoint(net, None, resume_path)

#Prepare dataloaders
test_images_dir = '/hdd/HENRI/goosynth/test/'
test_pickle_path = '/hdd/HENRI/goosynth/picklefiles/testpickle120.pickle'
batch_size = 16

#For GOO
val_set = GooDataset(test_images_dir, test_pickle_path, 'test', use_bboxes=True)

test_only = True
if not test_only:
    test_and_save(net, test_data_loader)

=> loading checkpoint './saved_models/gazenet_goo/model_epoch25.pth.tar'
=> Optimizer has different parameter groups. Usually this will occur for staged optimizers (GazeNet, GazeMask)
=> loaded checkpoint './saved_models/gazenet_goo/model_epoch25.pth.tar' (epoch 25)


/home/eee198/Documents/gaze-on-objects/gazefollowing/dataloader/gazenet.py:36: UserWarning: Enabling use_bboxes prevents this pytorch Dataset module to be used in the pytorch Dataloader module due to varying amount of bboxes in an image (Unstackable).
  warnings.warn('Enabling use_bboxes prevents this pytorch Dataset module to be used in the pytorch Dataloader module due to varying amount of bboxes in an image (Unstackable).')


Number of Images: 19200


In [22]:
predictions_path = 'predictions.npz'
metrics = calculate_metrics(predictions_path, val_set)

100%|██████████| 19200/19200 [10:59<00:00, 29.12it/s]


In [23]:
PA_count = metrics['cpa']
print('MSE: ', metrics['mse'])
print("Percentage Distances: ", [0.01, 0.03, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30])
print("Proximate Accuracy: \t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t"%
            (PA_count[0],
            PA_count[1],
            PA_count[2],
            PA_count[3],
            PA_count[4],
            PA_count[5],
            PA_count[6],
            PA_count[7],
            ))

MSE:  0.10772794595155237
Percentage Distances:  [0.01, 0.03, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
Proximate Accuracy: 	0.024	0.147	0.242	0.318	0.332	0.334	0.335	0.336	


In [8]:
def demo_random(npzfile, dataset):
    
    # Load predictions from saved npz file
    predictions = np.load(npzfile)
    gazepoints = predictions['gazepoints']

    # Get a random sample from dataset
    idx = np.random.randint(len(dataset))
    data = dataset[idx]

    # Load data from the sample
    image, face_image, gaze_field, eye_position = data['image'], data['face_image'], data['gaze_field'], data['eye_position']
    image, face_image, gaze_field, eye_position = map(lambda x: Variable(x.cuda(), volatile=True), [image, face_image, gaze_field, eye_position])
    gt_position = data['gt_position']
    image_path = data['image_path']
    gt_bboxes = data['gt_bboxes']
    gt_labels = data['gt_labels']
    
    # Draw gazepoints and gt
    im = cv2.imread(image_path)
    image_height, image_width = im.shape[:2]
    x1, y1 = eye_position
    x2, y2 = gt_position
    x3, y3 = gazepoints[idx]
    x1, y1 = image_width * x1, y1 * image_height
    x2, y2 = image_width * x2, y2 * image_height
    x3, y3 = image_width * x3, y3 * image_height
    x1, y1, x2, y2, x3, y3 = map(int, [x1, y1, x2, y2, x3, y3])
    cv2.circle(im, (x1, y1), 5, [255, 255, 255], -1)
    cv2.circle(im, (x2, y2), 5, [255, 255, 255], -1)
    cv2.circle(im, (x3, y3), 5, [255, 255, 255], -1)
    cv2.line(im, (x1, y1), (x2, y2), [255, 0, 0], 2) 
    cv2.line(im, (x1, y1), (x3, y3), [0, 165, 255], 2) 

    # Select nearest bbox given the gazepoint
    gazepoint = gazepoints[idx]
    #gt_bboxes = gt_bboxes / [image_width, image_height, image_width, image_height]
    bbox_data = select_nearest_bbox(gazepoint, gt_bboxes, gt_labels)
    nearest_bbox = bbox_data['box']

    # Scale to image size
    nearest_bbox = nearest_bbox * [image_width, image_height, image_width, image_height]
    nearest_bbox = nearest_bbox.astype(int)

    # Draw bbox of prediction
    cv2.rectangle(im, (nearest_bbox[0], nearest_bbox[1]), (nearest_bbox[2], nearest_bbox[3]), (0,165,255), 2)
    
    # Draw bbox of gt
    gaze_idx = data['gaze_idx']
    box = gt_bboxes[gaze_idx]
    nearest_bbox = box * [image_width, image_height, image_width, image_height]
    nearest_bbox = nearest_bbox.astype(int)
    cv2.rectangle(im, (nearest_bbox[0], nearest_bbox[1]), (nearest_bbox[2], nearest_bbox[3]), (255,0,0), 2)

    img = im
    save_dir = './temp/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    filename = 'out_%s.png' % str(1)
    save_path = save_dir + filename
    cv2.imwrite(save_path, img)
    
    return None

In [9]:
demo_random('predictions.npz', val_set)

/home/eee198/anaconda3/envs/gazefollow/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]


In [32]:
!python inference.py \
--test_dir='/hdd/HENRI/goosynth/test/'\
--test_annotation='/hdd/HENRI/goosynth/picklefiles/testpickle120.pickle'\
--resume_path='./saved_models/gazenet_goo/model_epoch25.pth.tar'

=> loading checkpoint './saved_models/gazenet_goo/model_epoch25.pth.tar'
=> Optimizer has different parameter groups. Usually this will occur for staged optimizers (GazeNet, GazeMask)
=> loaded checkpoint './saved_models/gazenet_goo/model_epoch25.pth.tar' (epoch 25)
/home/eee198/Documents/gaze-on-objects/gazefollowing/dataloader/gazenet.py:36: UserWarning: Enabling use_bboxes prevents this pytorch Dataset module to be used in the pytorch Dataloader module due to varying amount of bboxes in an image (Unstackable).
  warnings.warn('Enabling use_bboxes prevents this pytorch Dataset module to be used in the pytorch Dataloader module due to varying amount of bboxes in an image (Unstackable).')
Number of Images: 19200
